In [63]:
# Import Dependencies
import billboard
import sqlite3
from sqlalchemy import create_engine
import requests
import json
from collections import Counter


In [45]:
# Call hot 100 tracks billbaord API using billboard library
chart = billboard.ChartData('hot-100')

songs = []

for song in chart:
    
#     Add songs and artists to list for API calls
    songs.append(song.title)

In [46]:
# Call top 200 tracks billbaord API using billboard library
chart = billboard.ChartData('artist-100')

artists = []

for artist in chart:
    
    artists.append(artist.artist)


In [33]:
# Access token for Spotify API calls
# Enter your access_token given to you. Will import have to figure out a way to import from js

access_token = "BQCffk6PNl2jAPWa5kDZ6kefaNNnf_V6GQvXjkUAqAkFjNvA219Duz3CGXBE_F8dSuKjmcl5Pg1A6wnbKks3xK53UlVevHWxFGT1RM6NitsVmCk3F7xJoE5r9iHCSPBK6scqXHjVDmlliPwGSiEWKrr7-nwJqEQXi0Qj"

In [14]:

top_songs_json = []
song_ids = []
popularity = []
markets = []

for song in songs:
    
#     Spotify API search endpoint returns list of tracks matching track title in order of popularity
    endpoint = f"https://api.spotify.com/v1/search?q={song}&type=track"
    headers = {"Authorization":f"Bearer {access_token}"}

    response = requests.get(endpoint,headers=headers).json()
    
    try:
#         Apped first json object to list (The most popular track matching search criteria)
        track = response["tracks"]["items"][0]
        top_songs_json.append(track)
        
#         Append song_id for use in second API Call
        song_id = response["tracks"]["items"][0]["id"]
        song_ids.append(song_id)
        
        #  Append markets 
        market = response["tracks"]["items"][0]["available_markets"]
        markets.append(market)
        
    except: 
#         Alert for any track that was not appended to lists
        print("Song not Found")
        print(json.dumps(response["tracks"]["items"][0]["artists"],indent=4))


In [15]:
# Verify list length
print(len(song_ids))

100


In [59]:
# Count the number of countries in which each song is available
market_counter = Counter(x for sublist in markets for x in sublist)

# Convert counter object to dictionary
market_counter = dict(market_counter)

print(market_counter)

{'AD': 100, 'AR': 100, 'AT': 100, 'AU': 100, 'BE': 100, 'BG': 100, 'BO': 100, 'BR': 100, 'CA': 100, 'CH': 100, 'CL': 100, 'CO': 100, 'CR': 100, 'CY': 100, 'CZ': 100, 'DE': 100, 'DK': 100, 'DO': 100, 'EC': 100, 'EE': 100, 'ES': 100, 'FI': 100, 'FR': 100, 'GB': 100, 'GR': 100, 'GT': 100, 'HK': 100, 'HN': 100, 'HU': 100, 'ID': 100, 'IE': 100, 'IL': 100, 'IS': 100, 'IT': 100, 'JP': 100, 'LI': 100, 'LT': 100, 'LU': 100, 'LV': 100, 'MC': 100, 'MT': 100, 'MX': 100, 'MY': 100, 'NI': 100, 'NL': 100, 'NO': 100, 'NZ': 100, 'PA': 100, 'PE': 100, 'PH': 100, 'PL': 100, 'PT': 100, 'PY': 100, 'RO': 100, 'SE': 100, 'SG': 100, 'SK': 100, 'SV': 100, 'TH': 100, 'TR': 100, 'TW': 100, 'US': 100, 'UY': 100, 'VN': 100, 'ZA': 100}


In [60]:
market_dic = []

# Reformat dictionary for addtion to database
for key, value in market_counter.items():
    temp = {"country_code": key,
           "market_count": value}
    market_dic.append(temp)
    
# print(market_dic)

In [ ]:
# raw json
audio_features = []

# List to Load into db
features = []

for song_id in song_ids:
    
#     Call Spotify API to collect Audio features data
    endpoint = f"https://api.spotify.com/v1/audio-features/{song_id}"
    headers = {"Authorization":f"Bearer {access_token}"}

    response = requests.get(endpoint,headers=headers).json()
    
    song_id = response["id"]
    
    endpoint2 = f"https://api.spotify.com/v1/tracks/{song_id}"
    headers2 = {"Authorization":f"Bearer {access_token}"}

    response2 = requests.get(endpoint2,headers=headers2).json()
    
#     Append raw json to list
    audio_features.append(response)

    
#     Append each feature
    features.append({
        "id": response["id"],
        "popularity": response2["popularity"],
        "danceability": response["danceability"],
        "energy": response["energy"],
        "loudness": response["loudness"],
        "speechiness": response["speechiness"],
        "duration_ms": response["duration_ms"],
        "tempo": response["tempo"]
    })
    

In [ ]:
# print(features)
print(artists)

In [50]:
genres = []

for artist in artists:
    
#     Spotify API search endpoint returns list of tracks matching track title in order of popularity
    endpoint = f"https://api.spotify.com/v1/search?q={artist}&type=artist"
    headers = {"Authorization":f"Bearer {access_token}"}

    response = requests.get(endpoint,headers=headers).json()
    try:
#       Append list of genres for each artist
        genre = response["artists"]["items"][0]["genres"]
        genres.append(genre)
    except:
#       Print each artist not found in API call
        print(f"Artist not found: {artist}")
    

In [58]:
# Count the frequency of each genre for the top-100 artists
genre_counter = Counter(x for sublist in genres for x in sublist)

# Convert counter object to dictionary
genre_counter = dict(genre_counter)

print(genre_counter)

{'dance pop': 25, 'pop': 45, 'post-teen pop': 19, 'pop punk': 3, 'rock': 6, 'contemporary country': 16, 'country': 9, 'country road': 12, 'modern country rock': 5, 'hip hop': 14, 'pop rap': 18, 'rap': 25, 'canadian hip hop': 1, 'canadian pop': 4, 'k-pop': 1, 'modern rock': 5, 'vegas indie': 2, 'album rock': 3, 'mellow gold': 4, 'soft rock': 3, 'detroit hip hop': 1, 'g funk': 1, 'trap music': 16, 'anthem worship': 1, 'ccm': 2, 'christian alternative rock': 2, 'christian music': 2, 'christian rock': 1, 'world worship': 1, 'worship': 2, 'emo rap': 3, 'electropop': 3, 'etherpop': 1, 'indie poptimism': 2, 'indie r&b': 2, 'r&b': 4, 'uk contemporary r&b': 1, 'urban contemporary': 2, 'viral pop': 2, 'boy band': 1, 'alternative metal': 1, 'screamo': 2, 'emo': 1, 'brostep': 1, 'progressive electro house': 1, 'conscious hip hop': 1, 'west coast rap': 1, 'southern hip hop': 6, 'underground hip hop': 1, 'vapor trap': 1, 'hip pop': 1, 'country dawn': 1, 'reggaeton': 3, 'trap latino': 2, 'modern alte

In [62]:
genre_dic = []

# Reformat dictionary for addtion to database
for key, value in genre_counter.items():
    temp = {"genre": key,
           "count": value}
    genre_dic.append(temp)
    
# print(genre_dic)

In [64]:
# Connect to db using sqlalchemy
db_uri = "sqlite:///music.db"
engine = create_engine(db_uri)

# create features table
# engine.execute('CREATE TABLE "features" ('
#                'id string NOT NULL,'
#                'popularity integer,'
#                'danceability integer,'
#                'energy integer,'
#                'loudness integer,'
#                'speechiness integer,'
#                'duration_ms integer,'
#                'tempo integer);')

# create map table
# engine.execute('CREATE TABLE "map" ('
#                'country_code string,'
#                'market_count integer);')

# create genre table
# engine.execute('CREATE TABLE "genre" ('
#                'genre string,'
#                'count integer);')



# c.execute("DROP TABLE features")


In [ ]:
conn = sqlite3.connect('music.db')

c = conn.cursor()


for feature in features:
    c.execute("insert into features values (?, ?, ?, ?, ?, ?, ?, ?)", 
              [feature['id'],feature['popularity'],feature['danceability'],feature['energy'],
              feature['loudness'],feature['speechiness'],feature['duration_ms'],feature['tempo']])
    conn.commit()
conn.close()

In [26]:
conn = sqlite3.connect('music.db')

c = conn.cursor()


for market in market_dic:
    c.execute("insert into map values (?, ?)", 
              [market['country_code'],market['market_count']])
    conn.commit()
conn.close()

In [66]:
conn = sqlite3.connect('music.db')

c = conn.cursor()


for genre in genre_dic:
    c.execute("insert into genre values (?, ?)", 
              [genre['genre'],genre['count']])
    conn.commit()
conn.close()

In [67]:
# verify table creation
# result = engine.execute('SELECT * FROM "features"')
# result = engine.execute('SELECT * FROM "map"')
result = engine.execute('SELECT * FROM "genre"')
print(result.fetchall())

[('dance pop', 25), ('pop', 45), ('post-teen pop', 19), ('pop punk', 3), ('rock', 6), ('contemporary country', 16), ('country', 9), ('country road', 12), ('modern country rock', 5), ('hip hop', 14), ('pop rap', 18), ('rap', 25), ('canadian hip hop', 1), ('canadian pop', 4), ('k-pop', 1), ('modern rock', 5), ('vegas indie', 2), ('album rock', 3), ('mellow gold', 4), ('soft rock', 3), ('detroit hip hop', 1), ('g funk', 1), ('trap music', 16), ('anthem worship', 1), ('ccm', 2), ('christian alternative rock', 2), ('christian music', 2), ('christian rock', 1), ('world worship', 1), ('worship', 2), ('emo rap', 3), ('electropop', 3), ('etherpop', 1), ('indie poptimism', 2), ('indie r&b', 2), ('r&b', 4), ('uk contemporary r&b', 1), ('urban contemporary', 2), ('viral pop', 2), ('boy band', 1), ('alternative metal', 1), ('screamo', 2), ('emo', 1), ('brostep', 1), ('progressive electro house', 1), ('conscious hip hop', 1), ('west coast rap', 1), ('southern hip hop', 6), ('underground hip hop', 1)